## Connect to your Worksace

In [ ]:
# Librerías para conexión con el Workspace desde cuenta Azure
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    # Se intenta obtener el token por defecto desde el workspace actual
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # En caso de que NO funcionen las credenciales por defecto, se abrirá el 
    # InteractiveBrowserCredential (para autenticarte desde el navegador)
    credential = InteractiveBrowserCredential()

In [ ]:
# Cree una instancia cliente para administrar el Worspace
ml_client = MLClient.from_config(credential=credential)

## Preparación de los datos (Transformación)

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input
import mltable

# Crea un Dataset basado en los archivos en la carpeta de datos local
nombre_tabla = "Analisis_Sentimientos"
version = "1"

# Cargamos los datos desde el Asset de datos y los leemos como DataFrame
data_asset = ml_client.data.get(nombre_tabla, version=version)
tbl = mltable.load(f'azureml:/{data_asset.id}')
df = tbl.to_pandas_dataframe()
df = df[: 100]
df

In [ ]:
# Ver la distribución que tenemos de los datos
import matplotlib.pyplot as plt

Pos = 0
Neg = 0

for Sentiment in df["sentimiento"]:
    if(Sentiment == 'positive'):
        Pos += 1
    else:
        Neg += 1

plt.bar(['Positivas', 'Negativas'], [Pos, Neg], color = ['lime', 'red'])

In [ ]:
# Preprocesamiento usando nltk
import nltk

# Descargar stopwords y otros recursos necesarios
nltk.download('stopwords')
nltk.download('punkt')  # Para tokenizers

from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
import re  

stop_words = set(stopwords.words('spanish'))

X = []

removedor_tags = re.compile(r'<[^>]+>')

sentences = list(df['reseña'])
for sen in sentences:

    # Filtrado de stopword
    for stopword in stop_words:
        sentence = sen.replace(" " + stopword + " ", " ")

    # Remover los elementos de HTML (Que aparecen en los comentarios)
    sentence = removedor_tags.sub('', sentence)
    # Remover espacios múltiples
    sentence = re.sub(r'\s+', ' ', sentence)
    # Convertir todo a minúsculas
    sentence = sentence.lower()
    # Filtrado de signos de puntuación
    tokenizer = RegexpTokenizer(r'\w+')
    # Tokenización del resultado (Aplicando el rechazo de tokens descrito)
    result = tokenizer.tokenize(sentence)
    # Agregar al arreglo los textos "destokenizados" (Como texto nuevamente)
    X.append(TreebankWordDetokenizer().detokenize(result))

In [ ]:
# Filtrado de más StopWords (Definidas por el usuario)
New_StopWords = ['a','acá','ahí','al','algo','algún','alguna','alguno','algunas','algunos','allá','allí','ambos','ante',
                 'antes','aquel','aquella','aquello','aquellas','aquellos','aquí','arriba','así','atrás','aun','aunque',
                 'bien','cada','casi','como','con','cual','cuales','cualquier','cualquiera','cuan','cuando','cuanto','cuanta',
                 'cuantos','cuantas','de','del','demás','desde','donde','dos','el','él','ella','ello','ellas','ellos','en',
                 'eres','esa','ese','eso','esas','esos','esta','esto','estas','estos','este','etc','ha','hasta','la','lo','las',
                 'los','me','mi','mis','mía','mías','mío','míos','mientras','muy','ni','nosotras','nosotros','nuestra',
                 'nuestro','nuestras','nuestros','os','otra','otro','otras','otros','para','pero','pues','que','qué','si','sí',
                 'siempre','siendo','sin','sino','so','sobre','sr','sra','sres','sta','su','sus','te','tu','tus','un','una',
                 'uno','unas','unos','usted','ustedes','vosotras','vosotros','vuestra','vuestro','vuestras','vuestros','y','ya',
                 'yo']

for i in range(len(X)):
    for element in New_StopWords:
        X[i] = X[i].replace(" " + str(element) + " ", " ")

print("Textos sin StopWords:")
X

In [ ]:
# Creación del vector objetivo (Postivo / Negativo)
Sent = df['sentimiento']

y = []
for Sentimiento in Sent:
    if Sentimiento == "positive":
        y.append(int(1))
    else:
        y.append(int(0))
print("Vector de objetivos:")
print(y)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Preparamos la capa de embeddingsn(Predefinimos una cantidad de
# 5000 palabras consideradas como tokens
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)

# Transforma cada texto en una secuencia de valores enteros
X_train = tokenizer.texts_to_sequences(X)

# Acceder al vocabulario
word_index = tokenizer.word_index

# Imprimir el vocabulario
print("Vocabulario:", word_index)

# Si solo quieres las palabras, puedes obtener las claves del diccionario
words = list(word_index.keys())
print("Palabras en el vocabulario:", words)

In [ ]:
# Guardamos el tokenizador
import pickle

# Guardar el tokenizador a un archivo
with open('./data/sentiments.csv', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(X_train[0])

In [ ]:
# Crear una matriz para el Bag of Words (inicialmente todo a cero)
import numpy as np
import pandas as pd

bag_of_words = np.zeros((len(X_train), len(word_index)))

# Rellenar la matriz con las frecuencias
for i, doc in enumerate(X_train):
    for word in doc:
        index = word - 1  # ajustando el índice porque el índice en Python es base cero
        if index < len(word_index):  # solo consideramos las palabras dentro del límite del vocabulario
            bag_of_words[i, index] += 1

# Crear el DataFrame
df = pd.DataFrame(bag_of_words, columns=list(word_index.keys()))

# Añadir el vector y como una columna adicional en el DataFrame
df['sentiment'] = y

# Imprimir el DataFrame
print(df)

In [ ]:
# Guardar el df en una dirección local
local_path = ""
df.to_csv(local_path, index=False)

In [ ]:
from azure.ai.ml.entities import Data

data = Data(
    path="",  # Asegúrate de que el camino sea accesible por Azure
    type="mltable",
    description="Descripción de los datos para analisis de sentimientos"
)

registered_data = ml_client.data.create_or_update(data)

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input
import mltable

# creates a dataset based on the files in the local data folder
training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:sentiments:1")

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="InstanciaAutoMLTest",                 # Nombre del cluster de cómputo
    experiment_name="AnalisisSentimientosExperimento", # Nombre que le quieres dar al experimento
    training_data=training_data_input,             # Data / Dataset de input
    target_column_name="sentiment",                 # Columna objetivo de la clasificación
    primary_metric="accuracy",                     # Metrica de evaluación
    n_cross_validations=5,                         # Número de validaciones cruzadas
    enable_model_explainability=True               # Permite las explicaciones de las predicciones
)

In [ ]:
# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

In [ ]:
# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)